**LIBRERIAS**

In [59]:
import pandas as pd
import pyarrow
from sqlalchemy import create_engine
from datetime import datetime
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv
import os

# Cargar variables de entorno
load_dotenv()

True

**CONEXIONES**

In [60]:
try:
    # Conexión a SalesSystem (Facturación) - Comentada pero usando variables de entorno
    """
    salessystem_url = f"{os.getenv('DB_SALESSYSTEM_DIALECT')}://{os.getenv('DB_SALESSYSTEM_USER')}:{os.getenv('DB_SALESSYSTEM_PASSWORD')}@{os.getenv('DB_SALESSYSTEM_HOST')}:{os.getenv('DB_SALESSYSTEM_PORT')}/{os.getenv('DB_SALESSYSTEM_NAME')}"
    salessystem = create_engine(salessystem_url)
    """
    
    # Conexión a Warehouse (Contabilidad)
    warehouse_url = f"{os.getenv('DB_WAREHOUSE_DIALECT')}://{os.getenv('DB_WAREHOUSE_USER')}:{os.getenv('DB_WAREHOUSE_PASSWORD')}@{os.getenv('DB_WAREHOUSE_HOST')}:{os.getenv('DB_WAREHOUSE_PORT')}/{os.getenv('DB_WAREHOUSE_NAME')}"
    warehouse = create_engine(warehouse_url)
    
    # Validar la conexión
    with warehouse.connect() as conn_warehouse:
        pass
        
except Exception as e:
    print(f"Error al conectar a la base de datos: {str(e)}")
    raise

**VARIABLES**  

In [61]:
def get_mes_anterior(meses):
    fecha_actual = datetime.now()
    mes_anterior = fecha_actual - relativedelta(months=meses)
    return int(mes_anterior.strftime('%Y%m'))
periodo_cerrado = get_mes_anterior(1)
periodo_corriente = get_mes_anterior(0)
periodo_seleccionado = 202511

**CONSULTAS SQL**

In [62]:
priv_entities=pd.read_sql("SELECT * FROM priv.entities ORDER BY ruc ASC", warehouse)
acc_9=pd.read_sql("SELECT * FROM acc._9", warehouse)

**EMPRESAS ORDENADAS SEGUN ULTIMO DIGITO**

In [63]:
entities_1 = priv_entities[priv_entities['activo'] == True].assign(
    ultimo_digito_ruc=lambda x: x['ruc'] % 10
).sort_values(by='ultimo_digito_ruc').drop(columns='ultimo_digito_ruc')
entities_1=entities_1[['nombre_razon', 'ruc', 'usuario_sol', 'clave_sol', 'observaciones']]

In [64]:
entities_1

,nombre_razon,ruc,usuario_sol,clave_sol,observaciones
120,JMV INVESTMENTS E.I.R.L,20611097400,10813326,Jorge2703,None
90,NOVATEX GLOBAL S.A.C,20603078200,LAMPTUDE,Cesar2025,ERROR DE LOGIN - Modal de Chrome no manejado (...
33,JACQUELINE CORDOVA,10469541661,BILENTRA,opertatio,None
53,MINIER DE OLEO DICKERSON MARINO,15609947851,OFFOROAR,iambustek,None
98,IML NEGOCIACIONES E.I.R.L.,20605380451,75950584,Imlsaave03,None
109,MEESA METAL ESCORPIO S.A.C.,20608570811,09367746,Mechan20,BAJA DE OFICIO
58,GEOSSTRATOS GEOTECHNICAL SOLUTIONS S.A.C.,20522580491,QUITORKN,Fabio2022,None
114,CLARDENT,20609943611,47251373,Clardent3611,None
122,EMPRESA ROSS MARINE SERVICE S.A.C,20611822201,LATOOTAL,actroushe,None
117,ROMARK PROJECTOS S.A.C.,20610704281,41993294,13092023Ms,None


**DECLARACIONES PENDIENTES MES EN CURSO**

In [65]:
# Filtro de periodo
acc_9_filtrado = acc_9[(acc_9['periodo_tributario'] == periodo_cerrado)]
# Filtro de no declarados
acc_9_filtrado = acc_9_filtrado[acc_9_filtrado['numero_orden'].isna()]
# Filtro de columnas
acc_9_filtrado = acc_9_filtrado[['ruc', 'periodo_tributario', 'fecha_presentacion','_100','_107','_301','_145']]
# Realizar un join entre priv_entities y acc_9_filtrado
resultado_join = entities_1.merge(acc_9_filtrado, on='ruc', how='left')
resultado_sin_coincidencias = resultado_join[resultado_join['periodo_tributario'].notna()]

In [66]:
resultado_sin_coincidencias

,nombre_razon,ruc,usuario_sol,clave_sol,observaciones,periodo_tributario,fecha_presentacion,_100,_107,_301,_145


**PLANILLA**

In [67]:
#NUEVA ESTRUCTURA DE REPORTE PARA PAGO DE PLANILLAS DEL MES
'''	join	join	join		concat1	concat1	if tipo_contrato A TIEMPO PARCIAL horas trabajadas 120, default 240				
ide.ruc	entities.usuario_sol	entities.clave_sol	entities.alias	ide.numero_documento	ide.nombre	ide.apellido_paterno	tra.tipo_contrato	ssc.salud	ssc.pension	comisiones_afp.sobre_flujo	comisiones_afp.obligatorio
tipo_documento#
numero_documento#
apellido_paterno#
apellido_materno#
nombres#
situacion#
tipo_trabajador#
tipo_regimen_salud
fecha_inicio_salud
EPS/Serv. Propio
tipo_regimen_pension
fecha_inicio_pension
CUSPP
sctr_salud_ninguno
sctr_salud_essalud
scrt_salud_eps
scrt_pension_ninguno
sctr_pension_onp
sctr_pension_privado
rentas_lir
evit_doble_imp
'''
planillas=pd.read_sql("SELECT entities.alias, ide.numero_documento, CONCAT(ide.nombre,' ', ide.apellido_paterno), tra.regimen_laboral, tra.remuneracion FROM payroll.ide INNER JOIN payroll.tra ON ide.cui=tra.cui_relacionado LEFT JOIN priv.entities ON entities.ruc=ide.ruc WHERE tra.situacion='ACTIVO' ORDER BY entities.alias", warehouse)
planillas


,alias,numero_documento,concat,regimen_laboral,remuneracion
0,ALIAGA L,21141522,DONATO LOPEZ,MICROEMPRESA,1025
1,ALIAGA L,46482712,SEGUNDO BENANCIO AMASIFUEN,MICROEMPRESA,1025
2,CIELO,44780820,LILIANA MIRELY COCHACHIN,D LEG N.° 728,1500
3,CLARDENT,47251373,CLARA GISELA RAMOS,MICROEMPRESA,513
4,CONSUL CACH,07635569,ROBERTO GLORIOSO ARANDA,D LEG N.° 728,1025
5,CONSUL CELIZ,07618161,JOSE ANTONIO PESANTES,D LEG N.° 728,1025
6,ELITE,40691608,CARLOS ALBERTO CELIZ,D LEG N.° 728,1500
7,ELITE,46135144,JESUS MARTIN GIRATA,D LEG N.° 728,1025
8,ESPINO,40081708,MAYRA JANNELLE VELA,D LEG N.° 728,1025
9,ESPINO,40227069,GIULIO SANDRO ESPINO,D LEG N.° 728,1025


**RESUMEN SIRE**

In [68]:
ventas=pd.read_sql("SELECT * FROM acc._5 WHERE periodo_tributario=" + str(periodo_seleccionado), warehouse)
compras=pd.read_sql("SELECT * FROM acc._8 WHERE periodo_tributario=" + str(periodo_seleccionado), warehouse)
ventas_exportaciones=ventas.loc[
    (ventas['tipo_operacion'] == 17) &
    (ventas['destino'] == 2),
    ].groupby('ruc')['valor'].sum().rename("ventas_exportaciones")
nogravadas_destino_2=ventas.loc[
    (ventas['tipo_operacion'] == 1) &
    (ventas['tipo_comprobante'] != 7) &
    (ventas['destino'] == 2), 
    ].groupby('ruc')['valor'].sum()
nogravadas_destino_3=ventas.loc[
    (ventas['tipo_operacion'] == 1) &
    (ventas['tipo_comprobante'] != 7) &
    (ventas['destino'] == 3), 
    ].groupby('ruc')['otros_cargos'].sum()
ventas_nogravadas = nogravadas_destino_2.add(nogravadas_destino_3, fill_value=0).rename("ventas_nogravadas")
ventas_gravadas=ventas.loc[
    (ventas['tipo_operacion'] == 1) &
    (ventas['tipo_comprobante'] != 7) &
    (ventas['destino'] != 2),
    ].groupby('ruc')['valor'].sum().rename("ventas_gravadas")
ventas_ivap=ventas.loc[
    (ventas['tipo_comprobante'] != 7) &
    (ventas['destino'] == 4),
    ].groupby('ruc')['valor'].sum().rename("ventas_ivap")
ventas_isc=ventas.loc[
    (ventas['tipo_comprobante'] != 7)
    ].groupby('ruc')['isc'].sum().rename("ventas_isc")
ventas_icbp=ventas.loc[
    (ventas['tipo_comprobante'] != 7)
    ].groupby('ruc')['icbp'].sum().rename("ventas_icbp")
ventas_otros=ventas.loc[
    (ventas['destino'] != 3),
    ].groupby('ruc')['otros_cargos'].sum().rename("ventas_otros")
compras_gravadas18=compras.loc[
    (compras['tipo_operacion'] == 2) &
    (compras['tipo_comprobante'] != 7) &
    (compras['igv']/compras['valor'] > 0.17) &
    (compras['destino'] != 4),
    ].groupby('ruc')['valor'].sum().rename("compras_gravadas18")
nogravadas_destino_4 = compras.loc[
    (compras['tipo_operacion'] == 2) &
    (compras['tipo_comprobante'] != 7) &
    (compras['destino'] == 4),
].groupby('ruc')['valor'].sum()
nogravadas_destino_5 = compras.loc[
    (compras['tipo_operacion'] == 2) &
    (compras['tipo_comprobante'] != 7) &
    (compras['destino'] == 5),
].groupby('ruc')['otros_cargos'].sum()
compras_nogravadas = nogravadas_destino_4.add(nogravadas_destino_5, fill_value=0).rename("compras_nogravadas")
compras_gravadas10=compras.loc[
    (compras['tipo_operacion'] == 2) &
    (compras['tipo_comprobante'] != 7) &
    (compras['igv']/compras['valor'] < 0.11) &
    (compras['destino'] != 4),
    ].groupby('ruc')['valor'].sum().rename("compras_gravadas10")
compras_isc=compras.loc[
    (compras['tipo_comprobante'] != 7)
    ].groupby('ruc')['isc'].sum().rename("compras_isc")
compras_icbp=compras.loc[
    (compras['tipo_comprobante'] != 7)
    ].groupby('ruc')['icbp'].sum().rename("compras_icbp")
compras_otros=compras.loc[
    (compras['tipo_comprobante'] != 7) &
    (compras['destino'] != 5)
    ].groupby('ruc')['otros_cargos'].sum().rename("compras_otros")

conceptos = [ventas_exportaciones, ventas_gravadas, ventas_nogravadas, ventas_otros, ventas_ivap, ventas_isc, ventas_icbp, compras_gravadas18, compras_gravadas10, compras_nogravadas, compras_otros, compras_isc, compras_icbp]
resumen = pd.concat(conceptos, axis=1).fillna(0).reset_index()
resumen_final = pd.merge(
    resumen,
    priv_entities[['ruc','alias']],
    on='ruc',        # La columna para unir
    how='left'       # Usamos LEFT JOIN para no perder ningún RUC de tus cálculos
)

# Opcional: Reordenar las columnas para que el alias aparezca primero
columnas_ordenadas = ['alias'] + [col for col in resumen_final.columns if col not in ['alias', 'ruc']]
resumen_final = resumen_final[['ruc'] + columnas_ordenadas]


In [69]:
resumen_final

,ruc,alias,ventas_exportaciones,ventas_gravadas,ventas_nogravadas,ventas_otros,ventas_ivap,ventas_isc,ventas_icbp,compras_gravadas18,compras_gravadas10,compras_nogravadas,compras_otros,compras_isc,compras_icbp
0,10085917213,FERNANDEZ M,0.0,13094.90,0.0,0.0,0.0,0.0,0.0,1912.03,0.00,385.63,0.00,0.0,0.0
1,10105880818,BELTRAN G LE,0.0,5500.00,0.0,0.0,0.0,0.0,0.0,5445.81,441.02,24.80,28.58,0.0,0.0
2,10297073058,PAJUELO P,0.0,2500.00,0.0,0.0,0.0,0.0,0.0,3228.84,117.65,124.26,0.00,0.0,0.0
3,10406916087,CELIZ C,0.0,14820.89,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0
4,10456396572,LLANOS E,0.0,8300.00,0.0,0.0,0.0,0.0,0.0,9053.06,0.00,0.00,0.00,0.5,0.0
5,10726501306,ALIAGA L,0.0,172828.83,0.0,0.0,0.0,0.0,0.0,119011.19,0.00,2468.92,0.00,0.0,0.0
6,10779181575,TARDILLO C,0.0,7090.00,0.0,0.0,0.0,0.0,0.0,5241.73,520.47,123.80,2.08,0.0,0.0
7,15609947851,MINIER D,0.0,3500.00,0.0,0.0,0.0,0.0,0.0,3540.81,0.00,0.00,4.67,0.0,0.0
8,20544564685,LA FRONTERA,0.0,1727.13,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0
9,20548030529,RPM,0.0,1120980.47,0.0,0.0,0.0,0.0,0.0,7566.76,0.00,0.00,0.00,0.0,0.0
